# Tropical cyclone hazard assessment station report

This provides a summary of the TC hazard at a station....

In [ ]:
%matplotlib notebook
from __future__ import print_function, division

In [ ]:
import database
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
from math import cos, asin, sqrt
import os
import pickle

from matplotlib.collections import LineCollection
from matplotlib.colors import BoundaryNorm, ListedColormap
from matplotlib.lines import Line2D

import cartopy.crs as ccrs
import cartopy.feature as feature
import shapely.geometry as sg


from netCDF4 import Dataset

from Utilities.config import ConfigParser
from Utilities.track import ncReadTrackData

# Import widgets for interactive notebook
from ipywidgets import interact, fixed, FloatSlider, Dropdown, interact_manual
import ipywidgets as widgets

from IPython.display import display

import seaborn as sns
sns.set_context("poster")
sns.set_style("whitegrid")

def makeSegments(xx, yy):
    points = np.array([xx, yy]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    return segments

def colorline(ax, xdata, ydata, zdata=None, alpha=0.9):
    """
    Given a collection of x,y points and optionally magnitude
    values for each point, plot the data as a collection of
    coloured line segments. Line segments are added to the given 
    :class:`matplotlib.axes` instance.
    
    .. note:: Currently, intervals are hard-coded
              for plotting the central pressure of TCs. 
              [800, 920, 935, 950, 970, 985, 1050]
    
    :params ax: :class:`matplotlib.axes` instance on which to plot the line segments
    :param xdata: array of x-coordinates of points to plot
    :param ydata: array of y-coordinates of points to plot
    :param zdata: (optional) array of magnitude values of the points to inform colouring
    :param alpha: transparency of the lines (default=0.9)
    
    """
    colours=['0.75', '#0FABF6', '#0000FF', 
             '#00FF00', '#FF8100', '#ff0000']
    intervals = [0, 17.5, 24.5, 32.5, 44.2, 55.5, 1000]
    intervals = [800, 920, 935, 950, 970, 985, 1050]
    segments = makeSegments(xdata, ydata)
    cmap = ListedColormap(colours[::-1])
    norm = BoundaryNorm(intervals, cmap.N)
    lc = LineCollection(segments, array=zdata, cmap=cmap,
                        norm=norm, alpha=alpha)

    labels = ['No data', 'Category 1', 'Category 2',
              'Category 3', 'Category 4', 'Category 5']
    handles = []
    for c, l in zip(cmap.colors, labels[::-1]):
        handles.append(Line2D([0], [0], color=c, label=l))

    ax.add_collection(lc)
    ax.legend(handles, labels[::-1], loc=2, frameon=True, prop={'size': 10})

def distance(lat1, lon1, lat2, lon2):
    """
    Calculate distance between two points on the Earth's surface
    
    """
    p = 0.017453292519943295
    a = 0.5 - np.cos((lat2-lat1)*p)/2 + np.cos(lat1*p)*np.cos(lat2*p) * (1-np.cos((lon2-lon1)*p)) / 2
    return 12742 * np.arcsin(np.sqrt(a))

def loadTrack(trackId):
    """
    Given a track id, load the data from the corresponding track file.
    
    :param str trackId: A track id code that looks like "xxx-xxxxx"
    
    :returns: A :class:`Track` object containing the track data
    """

    trackNum, trackYear = int(trackId.split('-')[0]), int(trackId.split('-')[1])
    trackFile = os.path.join(outputPath, 'tracks', 'tracks.{0:05d}.nc'.format(trackYear))
    tracks = ncReadTrackData(trackFile)
    
    return [t for t in tracks if t.trackId==(trackNum, trackYear)][0]

def getTracks(recs):
    """
    Given a list of events, load the track data from a file and append to a list of track objects
    
    :param recs: A list of records, which include an 'eventId' field, which is the track id code
    
    :returns: A list of :class:`Track` objects.
    """
    tracks = []
    for rec in recs:
        trackId = rec['eventId']
        track = loadTrack(trackId)
        tracks.append(track)
    return tracks

def plot_tracks(gridLimit, tracks):
    '''
    Plot all of the tracks contained within the 'tracks' object
    
    .. note:: This _tries_ to intelligently determine if the mapped area 
              crosses the dateline (180E), which causes some problems
              in the `cartopy` routines. 
    
    :param dict gridLimit: Extent of the map figre to plot. 
    :param list tracks: a list of :class:`Track` objects to plot. 
    '''
    fig = plt.figure(figsize=(12,6))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines(resolution='10m', color='black', linewidth=1)
    ax.add_feature(feature.BORDERS)
    gl = ax.gridlines(linestyle=":", draw_labels=True)
    gl.xlabels_top = False
    gl.ylabels_right = False
    
    if gridLimit['xMax'] > 180:
        ax.set_xlim((gridLimit['xMin']-360, gridLimit['xMax']-360))
    else:
        ax.set_xlim((gridLimit['xMin'], gridLimit['xMax']))
    ax.set_ylim((gridLimit['yMin'], gridLimit['yMax']))
    for track in tracks:
        if gridLimit['xMax'] > 180:
            colorline(ax, track.Longitude-360., track.Latitude, track.CentralPressure)
        else:
            colorline(ax, track.Longitude, track.Latitude, track.CentralPressure)
        ax.hold(True)
    return fig, ax


def filter_events(events, category, distance):
    """
    Filter a set of evets based on intensity and distance from the location. 
    """
    extreme_events = [e for e in events if e['wspd'] > category]
    print(("There are {0} events that generate wind "
           "speeds greater than {1} m/s at {2}").format(len(extreme_events),
                                                        category, locName))
    tracks = getTracks(extreme_events)
    tracks = [t for t in tracks if t.minimumDistance([locpt]) < distance]
    extremedf = pd.DataFrame.from_records(extreme_events, columns=events.dtype.names)
    print("There are {0} events that pass within {1} km of {2}".format(len(tracks), distance, locName))
    return extremedf, tracks

In [ ]:
working_dir = '/g/data/fj6/TCRM/TCHA18'
configFile = os.path.join(working_dir, 'tcrm2.1.ini')

In [ ]:
def selectTopEvents(hazard_db, locId):
    """
    Select the top ten wind speed records for a given location.

    :param hazard_db: :class:`HazardDatabase` instance.
    :param int locId: Location identifier.

    :returns: :class:`numpy.recarray` containing the location id, location
              name, wind speed and event id of the ten highest events.

    """
    query = ("SELECT w.locId, l.locName, w.wspd, w.umax, w.vmax, w.eventId "
             "FROM tblWindSpeed w "
             "INNER JOIN tblLocations l "
             "ON w.locId = l.locId "
             "WHERE l.locId = ? ORDER BY w.wspd DESC LIMIT 10")
    cur = hazard_db.execute(query, (locId,))
    results = cur.fetchall()
    results = np.rec.fromrecords(results,
                                 names=('locId,locName,wspd,'
                                        'umax,vmax,eventId'))

    return results

In [ ]:
config = ConfigParser()
config.read(configFile)

db = database.HazardDatabase(configFile)
locations = db.getLocations()
locNameList = list(locations['locName'])
outputPath = config.get('Output', 'Path')
gridLimit = config.geteval('Region', 'gridLimit')
l = Dropdown(options=locNameList, description="Location")
display(l)

In [ ]:
locName = l.value

locId = locations['locId'][locations['locName']==locName][0]
locLon = locations['locLon'][locations['locId']==locId][0]
locLat = locations['locLat'][locations['locId']==locId][0]
locpt = sg.Point(locLon, locLat)
events = selectTopEvents(db, locId)

tracks = getTracks(events)
eventdf = pd.DataFrame.from_records(events, columns=events.dtype.names)

In [ ]:


fig, ax = plot_tracks(gridLimit, tracks)  
ax.scatter(locpt.x, locpt.y, color='k', s=20, zorder=100)